In [99]:
import numpy as np
import tensorflow as tf
import random 
import matplotlib.pyplot as plt
import scipy.io
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import pandas as pd
import re

In [100]:
# create dictionaries from the tokenized text
def create_dicts_from_tokenized_text(tokenized_text,vocabulary_size):
    words_and_count = Counter(tokenized_text).most_common(vocabulary_size - 1)
    print(words_and_count)
    word2id = {word: word_id for word_id, (word, _) in enumerate(words_and_count, 1)}
    word2id["_UNKNOWN_"] = 0
    id2word = dict(zip(word2id.values(), word2id.keys()))
    return word2id, id2word

In [101]:
#tweets_CDU1.encode('utf-8').strip()

#with open("tweets_CDU1.txt", 'rb') as f:

#    text_char = f.read()

#text_char = open("tweets_CDU1.txt",'rb').read()
#text_char = open("all_files1.txt",'r').read()

with open("tweets.txt", encoding="utf8", errors='ignore') as f:
    text_char = f.read()
    
    
text_char = re.sub(r"@", "", text_char)

In [153]:
# Generate the vocabulary.
#vocab = list(set(text_char))
#vocab_size = len(vocab)
#print(vocab)
#print("vocab size: {}".format(vocab_size))
#
#text_len = len(text_char)
#print("text lenght: {}".format(text_len))
#print(text_char)


# 1.st way to do it
tokenized_text = list(text_char.split(" "))
vocab = set(tokenized_text)

# 2nd way to do it - tokenized es, aber dann sind einzelne Buchstaben und Satzzeichen weg
#tokenizer = RegexpTokenizer(r'\w+')
#tokenized_text = list(tokenizer.tokenize(text_char))
#vocab = set(tokenized_text)

vocab_size = len(vocab)
vocab
print(vocab)
print("vocab size: {}".format(vocab_size))

text_len = len(tokenized_text)
print("text lenght: {}".format(text_len))
print(text_len)

{'', 'Boris', 'Bayreuther', '#EU', 'Outfit:', 'Faltencheck:', '"Meistersinger"', 'realitaetsfern.', 'Steuererhoehungen', '18.', 'Stimmt', 'solchen', 'nicht', 'Rennen', 'Langsam', 'Palmers', 'Manderla', 'intolerant', 'noch', 'Nachtkritik', 'Jahr', 'beschliet', 'meiste,', 'bettercallaxel', 'Familien', 'gestellt', 'BMI_Bund:', '#PDF', 'Direktkandidatin', 'Laut', 'Die', 'kamen', 'kahrs', 'Kommt', 'So', 'jreichelt:', ':', 'Platz', 'Hauptsache,', 'spdde', '&amp;', 'Hier', 'jetzt', 'Tourismusministerium', 'Wachsende', 'die', 'grundsaetzlich', 'Prozesse', 'wieder', 'Nuernberger', '#EU-ICT-Karte', 'das', 'Was', 'allen', 'Fachkraefte-#Zuwanderung', 'Erfolg', 'YouTube', 'wird', 'wiegelmann:', 'gemobbt', 'CDU:', 'neue', 'einer', 'Zukunft', 'den', 'Kloeckner', 'Gesinnung', 'via', 'aus"', 'Schwarz-Gruen', 'nicht!', '2017', 'bereits', '#Europabarometer:', 'Mindestlohnerhoehung', 'werden', 'ein', 'geprueft.', 'r#BTW17', 'Gisela', 'Text', 'nicht.][][][Damnits_Kathy', 'CDU,', 'unser', '.PeterWeissMdB:',

In [154]:
vocab_size

166

In [158]:
# Create dictionaries to switch between the indices of the characters and the characters themselves.
#char_to_idx = { ch:i for i,ch in enumerate(vocab) }
#idx_to_char = { i:ch for i,ch in enumerate(vocab) }

# Translate the text to indices.
#text_idx = [char_to_idx[c] for c in text_char]

# create dictionaires from tokenized text with a vocabulary size of 10000
#word2id, id2word = create_dicts_from_tokenized_text(vocab, 1000000)

# create dictionaries
word_to_id = { ch:i for i, ch in enumerate(vocab)}
id_to_word = {i:ch for i, ch in enumerate(vocab)}

# Translate the text to indices.
text_idx = [word_to_id[c] for c in tokenized_text]
#word_to_id

In [159]:
# Set the lenght of sequences we want to train on.
seq_len = 5

# Generate the dataset.
input_data = []
target_data = []
for i in range(text_len-seq_len):
    input_data.append(text_idx[i:i+seq_len])
    target_data.append(text_idx[i+1:i+seq_len+1])
    
#input_data = np.cast(input_data, dtype = np.int32)

In [160]:
tf.reset_default_graph()
# Create the TensorFlow dataset.
dataset = tf.data.Dataset.from_tensor_slices((input_data,target_data))

dataset
#dataset = tf.reshape (dataset,[193,5])
#added to make float 32 and get correct shape
#dataset = tf.cast(dataset, dtype = tf.float32)

# We do not train on batches or shuffle the dataset.
iterator = tf.data.Iterator.from_structure(dataset.output_types,dataset.output_shapes)
iterator_init_op = iterator.make_initializer(dataset)



In [161]:
# Get the input from the generator.
next_batch = iterator.get_next()
input_data = next_batch[0]
target_data = next_batch[1]

# Create one hot tensors.
input_one_hot = tf.one_hot(input_data, depth=vocab_size)
target_one_hot = tf.one_hot(target_data, depth=vocab_size)

# Initialize the placeholder for the hidden state.
hidden_size = 100
init_hs = tf.placeholder(shape=[1, hidden_size], dtype=tf.float32)    

In [162]:
# Build the model.
with tf.variable_scope("RNN", reuse=tf.AUTO_REUSE) as scope:
    
    # Set hidden state.
    hs_t = init_hs
    # Initialize list to save the hidden states and outputs of the sequence.
    hs = []
    ys = []
    
    
    # Initialize all weights and biases.
    initializer = tf.random_normal_initializer(stddev=0.1)
    Wxh = tf.get_variable("Wxh", [vocab_size, hidden_size], initializer=initializer)
    Whh = tf.get_variable("Whh", [hidden_size, hidden_size], initializer=initializer)
    Why = tf.get_variable("Why", [hidden_size, vocab_size], initializer=initializer)
    bh  = tf.get_variable("bh", [hidden_size], initializer=initializer)
    by = tf.get_variable("by", [vocab_size], initializer=initializer)
    
    # Unfold the RNN for as many steps as our sequence is long.
    for t in range(seq_len):
                
        # Read out the ith input and the ith target character
        xs_t = input_one_hot[t,:]
        xs_t = tf.expand_dims(xs_t, axis=0)
        ts_t = target_one_hot[t,:]
        
        # Compute the new hidden state.
        hs_t = tf.tanh(tf.matmul(xs_t, Wxh) + tf.matmul(hs_t, Whh) + bh)
        # Compute the new output.
        ys_t = tf.matmul(hs_t, Why) + by
        # Store hidden state and output.
        hs.append(hs_t)
        ys.append(ys_t)
        
# The RNN is done. 
# Save the hidden state for feeding it to the next sub sequence.
hs_remember = hs[0]
print(ys)
# Compute the softmax of the very last prediction for sampling.
output_softmax = tf.nn.softmax(ys[-1])
# Compute the loss of all the outputs.
outputs = tf.concat(ys, axis=0)
print(outputs)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=target_one_hot, logits=outputs))

[<tf.Tensor 'RNN/add_2:0' shape=(1, 166) dtype=float32>, <tf.Tensor 'RNN/add_5:0' shape=(1, 166) dtype=float32>, <tf.Tensor 'RNN/add_8:0' shape=(1, 166) dtype=float32>, <tf.Tensor 'RNN/add_11:0' shape=(1, 166) dtype=float32>, <tf.Tensor 'RNN/add_14:0' shape=(1, 166) dtype=float32>]
Tensor("concat:0", shape=(5, 166), dtype=float32)


In [163]:
# Optimizer.
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
training_step = optimizer.minimize(loss)

In [ ]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(10):
    
        # Set time step counter.
        t = 0
        # Load the dataset into the iterator.
        sess.run(iterator_init_op)
        hs_remember_val = np.zeros([1,hidden_size])

        # Go through the dataset until its empty.
        while True:
            try:
                # If we are in the first time step intialize the hidden state with zeros.
                #if t == 0:
                #    hs_remember_val = np.zeros([1,hidden_size])
                
                # Feed in the last hidden state.
                # Read out the loss value for printing.
                # Read out the hidden state for the next forward step.
                # Do the training step.
                hs_remember_val, loss_val, _ = sess.run([hs_remember, loss, training_step], feed_dict={init_hs: hs_remember_val})
                
                # Increment the time step.
                t += 1

            # Stop if iterator is empty.
            except tf.errors.OutOfRangeError:
                break
                
          
        # After each epoch we print the loss value.
        print("Epoch: {}, Loss: {:f}".format(epoch, loss_val))
        
        # The main validation procedure we will use is sampling a text from our model to see how good we 
        # approximate the original dataset.
        
        # How many words would we like to sample?
        sample_len = 20
        
        # Get a random starting sequence from our training dataset.
        start_idx = random.randint(0, len(text_idx) - seq_len)
        seq_idx = text_idx[start_idx:start_idx + seq_len]      
      
        # List to store the characters sampled by our model.
        sample_seq_idx        = []
        sample_hs_remember_val = np.zeros([1,hidden_size])

        # Sample as many characters as we would like.
        for n in range(sample_len):
            
            # To feed the starting sequence into our model we first need to put it into a dataset.
            # As we do not compare anything here we need some fake target values.
            fake_target = np.zeros([1,5], dtype=np.int32)
            sample_dataset = tf.data.Dataset.from_tensor_slices(([seq_idx], fake_target))
            # Load this dataset into the iterator.
            sess.run(iterator.make_initializer(sample_dataset))
    
            # Now we need to read out two things. The softmax output for sampling a character and the hidden state to
            # feed it in again.
            sample_output_softmax_val, sample_hs_remember_val = sess.run([output_softmax, hs_remember],
                                                                       feed_dict={init_hs: sample_hs_remember_val})

            
            
            # Sample a character from the softmax output distribution and append it.
            sample = np.random.choice(range(vocab_size), p=sample_output_softmax_val.ravel())
            sample_seq_idx.append(sample)
            # Update the start sequence for sampling the next character
            seq_idx = seq_idx[1:] + [sample]
        
      
        # Print sample.
        sample_txt = ' '.join(idx_to_char[idx] for idx in sample_seq_idx)
        print('----\n %s \n----\n' % (sample_txt,))       

Epoch: 0, Loss: 4.881181
----
 um Bayreuther CDU an Besucher gestellt der Meistersinger Zukunft gruenen Besucher Wagners Boris Urlauber der solchen Kloeckner Europabarometer stellt der 
----

Epoch: 1, Loss: 5.546035
----
 Meistersinger Tourismusministerium auf meiste primitiv Boris Wir zu So Palmers mal allen via Wahlkrampf nach BayreutherFestspiele Fachkraefte primitiv um Direktkandidatin 
----

Epoch: 2, Loss: 4.752392
----
 r Pflege bettercallaxel die gestellt BMI_Bund Fachkraefte Jahr jreichelt mit kahrs Download primitiv hoffentlich Spa Kloeckner Bundeskabinett amp helfen Selbstgerecht 
----

Epoch: 3, Loss: 5.130781
----
 Mindestlohnerhoehung wird Viel Antraege zu meiste Erfolg gruenen das Sie sicher Selbstgerecht der grundsaetzlich 3 diesem das kahrs Boris werden 
----

Epoch: 4, Loss: 3.605276
----
 vermeldet Meistersinger in Bald Zuwanderung noch jreichelt allen drei kahrs 2017 nicht positiv Gisela der So Manderla 2017 intolerant Prozesse 
----

Epoch: 5, Loss: 2.978689
----


Epoch: 0, Loss: 2.288059
----
 ea  diruun FeSnier uannnn wgrnte?ytilftrer L,eeerdine H afubate  ieanngeer Fangla #nninaiuenernienel tean?inenge  Ninmene e ?uerergrernFer erreen  min v?l ?mnuegnn,, agiman agernFeninfn Wier slgnFAgtn 
----

Epoch: 1, Loss: 1.804944
----
 ei Fmmerel?Fer Fel lerera?ae? Fafer Fepe e: hil  ?r lis Fium escee  macaen HFelienhender aer n?u get en uerlh? Hienietserdun wet n? Hie  unstr Fie n?enck ser lel I? efer Ficier  aahen Falin,gtaacv l?. 
----

Epoch: 2, Loss: 2.072745
----
 ae. :Ftderhinuer EindRn dit 18.uW: W8sden N8?sin wH"m18 S8r 18.k[? 18 1W7r 8.s1M i8-ger 18.ker WPrdtn der 18ndWP 18 1g  ina nt Werf1n 18r 18sWir WAnder U8b 1? S8aderume  ien Mamen wer 18o WP SHKann un 
----

Epoch: 3, Loss: 1.868061
----
 1asi: Si de, r.wind ! b laim n Sin 1n miit leachhe @ungnn fanvin Fusd rerelt. Lilg:ntVwel fn H ?]emst eermen n=timd gr Eiruchen 1et WichenKong 1in nn .wisgen Jamikn angener serflir gi fer wen wicaFuet 
----

Epoch: 4, Loss: 1.925632
----
 h Wung fie Gasmimie dier Fimita nairkal KunsFe La Wi= ala awin SWat? ?eer WWfmer des vesReb der Ee hhrFar werd h amir! Wami D? eats ESmenerulala Get nacafa ier iet wir Fr des naenrg "Werl ieamar 1? do 
----

Epoch: 5, Loss: 1.861674
----
 eN unein. ESxest ir  imitentu " amPstuenefn in  Mimat Fouruss 1rh rober Saeinstdun  ToWet Fer wisimil: nichkinintlieities Wain din  wum.mit une hner  ind sbeneen" Hichtiabknhuab rnner upg under uei 1M 
----

Epoch: 6, Loss: 1.768264
----
 s hisker Weicht!Kn ua der Witiriamter urhink !gurhun in uan ietin. na ukineu dierer Wch un Kintser Ffehlabee Fim kind read: Muirkr dah mabt oirtin @18m ra "chergeen wilin wamit, mchtkele  womin. Wectk 
----

Epoch: 7, Loss: 2.081336
----
 mine hrmen volchuteiln hntoraman ruenkbarhis  laelterenmenhr Lin der int #Prung litBun Tohdutiunt Sicher oanfu hamn  vien n ndra ned.l mahnermn dnern delren der ies n aahtichen  @Paft an d,  anchera ? 
----

Epoch: 8, Loss: 1.534197
----
 : Eut her-hnbSt beiftieit. nn in der 18nen tin. itsen des ir it unt in meimder viintg her ien Haurtoeflngennnin lia Knen ie  Eeimititn mesak gem geien .Minnchn. ierei der Anber .in  isuele: vis Aberne 
----

Epoch: 9, Loss: 1.435708
----
 ichind hirzenmieg 1n. ialan Sen 18sahtin anfinganiungen Pialen dinbanfanren grhier Faha det Pbe: Fifohangebfa de  Pifherunn nalin axt  einialanen gne geamin in ae? ant  Patknbeht!  minitit! nacht! mhn 
----

Epoch: 10, Loss: 1.713243
----
 nd di ii gn gie gr Gitgn Snes der  Koen gnent Sne tien gn Stiman Sber Tubit Fichun Fatin gn de  Eir inukn Sie ent 18 Histheneng get ubtden der @Kubt gnereter ger 18 viag !amir Fn: 18s 18xin geun Stiut 
----

Epoch: 11, Loss: 1.762047
----
 ah kramenthear Ei .ita hreit in de in gesiet ri: den Eitensaenes der 1n. nir uets  @naetstist Fasin  Jan Ban Bar unter  Sosers Sch neum urneist rit nite vafh hoe de: Eirant Gis nien det 18a Wei ges ai 
----

Epoch: 12, Loss: 1.860812
----
 : Licheroch rech efing  Wachlar Finueterhon din mising miegest riutkander sichlea nerit aaaut: Lahrer ain soerneitit Siteistonig der mia telitia hia uagte Waffer  in ieefne: Wif ii. wnegentenbet uns r 
----

Epoch: 13, Loss: 1.659803
----
 !riam nam nia #in susala ienen Sesachamtin Spa 18a Wer 18czt?enge Seiomeinismmin ubderbagteim Landen Wnseroenden Spiunschichen Sit lisker ium nicht! Wa  u=ier ALcaklien #inir der abamth  @fo guschem   
----

Epoch: 14, Loss: 1.692321
----
  salaen nicht! Washekoscauneilae deun Wa Istampd &al :ir Tan itsmir wir in gn ,in smen wisien Ger, geaun ioschl: Wauchampin " Sonbet Loer Eamist am win fin, naen rielkan en hWiche: Gisirien G_Walaliw  
----

Epoch: 15, Loss: 1.697626
----
 it  Wa aefo1n  ir prden  @xorberSerfth Wio n urusmet Siramg inkan uschan in all: , sich n Hay winden insin sie s Lailt Wae w Wen Fber Feithvcan uthnn? @kermutchr! inser JarZunmet  Wireut vir musoeg ge 
----

Epoch: 16, Loss: 1.456457
----
  Snsierichennin Famlernongey un Sssuninken Was veerlitier lihkerennun  @Uuft via Eu Ler spder und n unis_ber upa vor iul neun mit sotsnien gerllatkn dni neun bat Toung Nmehketamit ver belahe: Sif #Pel 
----

Epoch: 17, Loss: 1.696989
----
 n an hesMeb richen Ln der 18. War uechliabtr Wahn Manelt nninn Wae wer Pelentsmit enten?uen ners Srin geruntsneber Tbir warde gept pieler ner er Katzl der wicheitinndn  Stunleen ei \n. nichun din sien 
----

Epoch: 18, Loss: 1.697832
----
 neu gin der CDU prsahgn ber Ctanern &e Minhln destener han uncheiresllienen Wi  spoeng: Wa @werder Ansing aer stsitieign an wer 18. Kut palahn get nis Tamps weloAnund  un ierschennine Ien #EU pitachta 
----

Epoch: 19, Loss: 1.027246
----
   @belmet. @Patkra heften Men nelalit meubntin eis koalit vorbel #EDlobd alier nnsil? IIstllit korrint#MDer samilia -we @netalit der uther wicher Gndere Mochknen Anelich !estt hieln Facher Lahinbelman 
----

Epoch: 20, Loss: 1.252708
----
  3. mubalocs Rei Kut  ver @beltger beronn: Eun drunner \erdeu Sieket ninnen Sp wer nie rualien nech  @iekoue  nen @nalien ber 1v  ubsfehrnin nerlntsm Ein uen der und n.en ger in der 18. WP fuer Famili 
----

Epoch: 21, Loss: 1.300344
----
 er  Laelt Mun Fastharennunner  Haur Sacds Lan wie nie Zufu ber Sr Gv.a @weisivra fst-Wir Eafongenut v. inder werder ut. @beld: ._scha,  @m iex WP fu mithe @eimnt, WP Fniste, La @CMoentl. @CIunber Wr i 
----

Epoch: 22, Loss: 0.755693
----
 nuer Wahh  Sir Piatker ner kar hn her 18. WP fuer Familien get n? Hier Jalliaben hn  Jalilaen ner naninnen  Haur  aninse den grianen Lis Rettlabekvea Pitzelfe  wir ,ia v Jungiraninien nnd nia Bor Baxi 
----

Epoch: 23, Loss: 0.948605
----
 Soe tlandenzn die sersift nicher intillaelt acheneln: #iuemttundingeru gnder  goaut alitkraubm sech nMan Poeer  aunisamalngnm liane! Win ltaup? virlited viatkneun Fanden Sen eest hex  voeift mamesta h 
----

Epoch: 24, Loss: 0.528845
----
 n ein Spr dia her inier Felllinnen niu nia @bed vie dre \nienen Speustinich nei  Mauschachenhn Mehn ber C8. WP Nuer Faxth bef nicher Grselern. Linge Unescha, unen Steumit debent  Eullaekanimnt.mibet F 
----

Epoch: 25, Loss: 0.313106
----
  ban iet mit stauer  neisteall: Wachker Tmxtk aeln Mahrerchaettnecht. IIs  ardel gneln: #Eulier ietzn die en Famein: mh wersunder winden Wahllt inn lier besterinet Jichn: Way kraubbt holfo @iesenbenen 
----

Epoch: 26, Loss: 0.589241
----
   Haup Peftenttin @BayreutherFan grieftla ber ueu Eer Eirien Hah  Spe " d. werdin der mekel Fellin. . uellnen @beut voaungem ber WPufter Die Zuku @ir @bem Aberein dir Doun Sta muthlaekn  in ztampr: Wa 
----

Epoch: 27, Loss: 0.691363
----
 lnenten miehstonninnen SpaupreatherFestPahte seia @wa @wim vi .#PfpTech nounn in olauelt,  @fotstbeln: Eifett  Eif @bereit  @DFebst. KoNn Pr werFelmant: Wa wktie, win die melt, wind gn eistersinthn ge 
----

Epoch: 28, Loss: 0.436280
----
 ampf. Viel Soungis dri Eg eichenenng dit motan? in wira #EUrputa EFante Wachsernbetserein: WPrfuen Spr Erannen Sie stieiti \efstech erung nSteltrelachend n I@ @wanis Fan wirden Antliexeln d der deu go 
----

Epoch: 29, Loss: 0.125220
----
 ht, intolerant und realit enefn: #Ein er ueber decht!olnu @PDF betze " .achtkritik z-aruen Spd er uefungo Stiuer Bandersabei ho bestler .ahtse "ie stemiti ieisten das meita  winden Spruechen, Eafomnte 
----

Epoch: 30, Loss: 0.320884
----
 in: Wagners Meisterfieh necht ,ists meinge "Mehstericht, in er Fanlfun" Bu, dreungen Meister_ csin nin ge Min unhen : Wacherhen" voeIs "Weristachengen Ser pemalt: Wagn roer stsun Pichkn en Ba wititich 
----

Epoch: 31, Loss: 0.323988
----
 s kamen Linden Win uba kra nKen  isschrh, Ber sehte " inabs ,am komtie getser  Manistir enssmebet  iomet Micht tin dtr innde  Wagisamae un Wir utiele: Wagners Meistersinger. Wachse sin der inn er eise 
----

Epoch: 32, Loss: 0.119709
----
 YouTube  Felllehe: .ts Kalp: Lind voan His Ka lahen  inalita Fistcher 1usinn nis Laen Ste #bp @bechel" vin.d rissch 3. Prbschanesta hettnecs  Vien ho-IIst voriftkoeln Sie me  @spn.btin ge d: FastRebe: 
----

Epoch: 33, Loss: 0.611523
----
 solchelnen des halennun un  iestoer #EU maspre foed n an ze scheldte #Euwinmeld dellagn Mochteriuthnet Beristsanisneckon Gesch jbst die dt  seutsv. .Bun Si den Absaeite  Mehts=Selbstgerecht. irhoue an 
----

Epoch: 34, Loss: 0.088004
----
 poberfutBunderaninaen ber CDU, Kochtiange schlitameffing masistenobn gerin Poen Struer Sin en Spr achtemet an zierhenden Sinberh, ube Zukanber Seu Eut Twer_salahn an wierheis MIIItioWa  htienfe, Min S 
----

Epoch: 35, Loss: 0.086176
----
 erden Giser Fusaklat. #Landen Sie Zukunft der #Pflege" 1isken in dre foe, Fastkerhin dis lickt! WPst Mellnben get uedt Im Plehtern Nauhukampie Kabmielin din erher Fahtkramit v. Platz 3.  @Potomaker: S 
----

Epoch: 36, Loss: 0.058115
----
 Palmenze #DF gese bie hopfn Min Zprungb Soe msicher in @simelt Mach  Nisana Mam nerNach @ochun Sp besta den Sie pre1ft Wichtneis Mahr: Longsammibt  @mollterW initsto fett voelland n inlchen in der 18. 
----

Epoch: 37, Loss: 0.369997
----
 stericher#bn dis heimift re matze den  itspreinedt der Eir pieln: Warahtititit reit vneln Botinit Meldetsmefitit die Zukungesttimititit  @CNtsie dit teritl amt prueg  iosec Mind: Babli aener Uulangem  
----

Epoch: 38, Loss: 0.585235
----
 ldetgerechnieittenlinktern in dee 18. @Kut din #Bun @rUellten Fest dia @winden Sin pr ies  berolltaneint die mat nbeder Wahn Spelmit-soben ein ,  @MDU, "oelle "Meht mosmiber  Hauftercch @opoelt der  M 
----

Epoch: 39, Loss: 0.169559
----
 n niangt Soeistsacher nicht.][][][@Damnits_Kathy Kommt hoffentlich noch @outelit @kotomere din Abtun? Hier utsucb St ber liakgr Spibstsamber nicht. Geschr, .@mitititin. das meianbe gerouchenent\n @XI- 
----

Epoch: 40, Loss: 0.067973
----
 skned genichtfele voerueghuefin din det 18. WP fehr Familien getan? Hier unser Faltenchechenren @neuftin vie hophenten grichtinieio Stimit Biriet Menden Sie premptin @bes Minkt=enn: Uamokchln: sthder  
----

Epoch: 41, Loss: 0.016879
----
 gemobbt  @CLinkonenn nach ,oen Steue ma Despchoed  uRa @welmekn @bein undin dmelaber Dicht, Wichkraenn: Wagn Sie Zutu grimitin.ht uestlahn get lianner Sin Zukunft der #Pfleetin diestP mierhon,e @tpsie 
----

Epoch: 42, Loss: 0.110818
----
 -#Zuwander @rounger Pfoenlingen gerschlla Jurd geanbe  Dinntan ers achte Jun isstolit &rmpfte,  Gestlahechen, Erroeh  aelftel @,edet Wagnn sanfe gesol? Hier unser. Wan sacherten di  #Pfteg "Wich aopan 
----

Epoch: 43, Loss: 0.090779
----
 ueu Ber seroeber Tusrhtinit  Deromitaloe: #EUrpalahn Min nir Eur Toueltinite  @Grung aete  @Jihkvt iet  @Jelster_in der Bun nich a ieite deis Mehrielt Bin der #BU rerlanken ner Bra @welttinden Uisere  
----

Epoch: 44, Loss: 0.052000
----
 cht! Wasfhen Sieterbei spd det licht: Was haben wir in der 18. WP fuer Familien getan? Hier unser Faltencoebs hreungen gest alla her" in @er ier lach noenget Way sMalitaestin @voungen geuch @Nisabahen 
----

Epoch: 45, Loss: 0.058419
----
 Mechlitneckon in der 18. WP fuer Familien getan? Hier unsen inedte #Kunge_Unien Sprueg nicht  iestsamin viruet alletzu deu #Bayreuther Festspieln: WP wercsechla zensellandes Reistech noch @Nindet Minn 
----

Epoch: 46, Loss: 0.047460
----
 -ICT-Kalt, Klitkv. Ssn all kande ges PrlinndnBscht, iet Uaue @relllia @kotoulden St  @rFestanin vie ssacherstlaers Solieltaen Abschlmelnun Sie aef zsalihaen riniestechre Mun dveutlun dis kraestldasen  
----

Epoch: 47, Loss: 0.093918
----
 ht. [][][@Damnits_Kathy Kommt hoffentlich noch @Nisalahe: Links=Selbstgerecht, intole: #EU poamt#MLinitaen in @XImGBset Minag wers Mahrl Sollt aMDe: "Mehn sundin  iaut MaBtkunden ei Prho Mahe toolle z 
----

Epoch: 48, Loss: 0.069884
----
 l d, wGamelt der gruen em toengen niotan Sie grlin enetent darinaehe: Unsere Nacht! ies hamel eindin @romatdestUn Spa prpittiser En des Rerachtopote #wander ue weruen Sperpen dpr mal warde  @sdRe sich 
----

Epoch: 49, Loss: 0.419565
----
 ut in der WP iner Fbelteliti  @Juhsen dt ued riam wor in en gelintor, ag Nubel gn anberges"bel @kahr PUhr: Stersemalban Bellag  Gesebali: Spimmt rochtaeisterlingen dir lichla in der 18. WP fuer Famili 
----

Epoch: 50, Loss: 0.161123
----
 en getan? Hier unser Faltencanbexer ningnndt un @iche: eindin ei Pin: n Hiemen Steuet Wachtkriunhnit urhelt Wachets " in der 18. WP fuer Familien getan? Hier unser Faltknnen stadin Ourf mma @welt,  @M 
----

Epoch: 51, Loss: 0.012235
----
 Michers @Pdestlohnn Sie net lerN Muisaen itsung lich foung EU-grsam?its nicht, intolerant und realitaetsfern. Ein stanker Eur laat ar ParNanhen inhrn an Muca @windin Sie krimitiv. Lin s=Selbstgerecht, 
----

Epoch: 52, Loss: 0.012508
----
 e sunger Winks=Spibst!ergellPhn der Eurolaxen "poung  Schmmt  Jurisgealft\n @Nahne: Ste melmalbrn hesfs malmn Sit mit ,ereich nosnner schersinien  DoPrstacheft Bor. acherg Mingen dir wer werden Sie kr 
----

Epoch: 53, Loss: 0.067895
----
 ramitkrichen unsonge_Et der #Plmober der #Pa wer mesalien hr ange ge fut-mriant: So berhlin Ba @spersinger  Hauptsabed . WPlaetzl #ruwan wis 1berger unnen ein Sparner unnentst ung mit uer loanger  @Pu 
----

Epoch: 54, Loss: 0.028655
----
 ubauber Er acheGr: Lochun  Europitanden #EUromitariet erscheiss"Win knechech nochschlibis nier nmut  ietalit Eeristlabickatsnerhen un en wierium Per karSpaber wirdin Siemprtimicht! intkrrien wir en ei 
----

Epoch: 55, Loss: 0.015111
----
 bbt  vie @ndcs dolle der Europaees Mahtsem icht.][][@Panes La, kander Fastsilinn: Buy spaelft dot  Pachen Spruechektsfftiettin Mend gemolit eit scheft der llaktsuchenesscbelt Mahp: Lochker Tlxa @warde 
----

Epoch: 56, Loss: 0.027101
----
 en Gesinnung  Stimmt nicht! War hacheneStem Eundets Maer  @miretschlieer Steuer Wahlkaben spdpieitiun: #Europabarometersunger wirdeg mobeue sindisschaer  Lahpr Sch hechteuesen @nech nossteesMabkrach n 
----

Epoch: 57, Loss: 0.004124
----
 sache, einer wird geunger MindestSchen um Platz 3.  @Potomaker: Schwarz-Gruen im rot-gruenen Outfit: Julia Kloeckner stellt Boris Palmen Siener Wahkkramit voche " poutstiall: #Eurosabalter  Haukskm :  
----

Epoch: 58, Loss: 0.052840
----
 cht Tibek hrinin gneis Lohh  ellin zlinnen gersungein der CDUNschen in di  @spon. Ginen sichln eis  aun suchlltz_ichen in der mesalite  Lunks=Seln t Wic kaben wirhin anDin vomwinhen g sien wer in zlie 
----

Epoch: 59, Loss: 0.019101
----
 nks=Selbs grpriei n allhen 1in der 18.  vit sold: und der liann: Sie primitin zi   LanissUch neG sin ier laats=malg  ia De zlin Bum Preraub in der 18. WP fuer Familien getan? Hier unser Faltenchecd: , 
----

Epoch: 60, Loss: 0.011379
----
 ,  @BMI_Bund: Fachkraefte-#Zuwange Wag srernen nicher 18 Plahnbet  Wif @spodet richtam.licher" n ausunn  Dahpr Lohnn mieher Lings LWin Machktae "it die  @Mehrsuecklehn ut ies nbche: Wastsaben Men @Bal 
----

Epoch: 61, Loss: 0.020064
----
 ommt hrfoeht gelingers Pohrsemaln dir Antolitan Sin interesegen git #Europabar werden Steuets"Meistersinger" eomn: Sof ues_Bames Langs=Sffen Sie zen Spr aensem linken Geter Faltenchecknko Diemabesten  
----

Epoch: 62, Loss: 0.011175
----
 gerecht, intol? Haer unser Faltencheck: #Diwiet dis Bomel werrie Mebs haeferun  iestobereussinisien gntaleraet Minde  Gerellan?  @Clinnon Bis mela Berrelche " @rungelmaner Wagners Meistersinger  Haup# 
----

Epoch: 63, Loss: 0.300900
----
 rger Prozesch @Nind  vindenmabirgelt , Falliet nber @Hiuet rich : Son achln aeisteder Unserbete zo he  @Pllnieit Sor Barer FahDkramit vo zu P poerg lach iosn Ser Biser wird gemob,  #EF @Mdin om werden 
----

Epoch: 64, Loss: 0.008051
----
  inssseer, MdSr umben des 18. WP fuer Familien getan? Hier #EU positiv gelm Pamen in din #PDF @checsubt: .@Peter Fehsanden Uuthkaeck Hoankander Wihlkan nn.nnsunge_Uninn zit pomen eit I8. WP nuer Fahp  
----

Epoch: 65, Loss: 0.009194
--------------------------------------------------------------------------